In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import collections
pd.set_option('display.max_columns', 500)

import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import transformers

import nltk
import re


from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from transformers import *

plt.style.use('seaborn')

MOLDEL_NAME = 'roberta-base'
MAX_LEN = 312

In [2]:
df_train = pd.read_pickle('../data/train/raw.pickle')
df_train.head()

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed,doi_cites,cites
0,hep-ph/9902295,Michael Kraemer,Mark E. Hayes (University College London) and ...,Heavy-Flavour Production at HERA,"LaTeX, 21 pages, 13 Postscript figures. Summar...","J.Phys.G25:1477-1493,1999",10.1088/0954-3899/25/7/332,"CERN-TH/99-30, UCL/HEP 99-03",hep-ph hep-ex,None,We review the theoretical and experimental s...,"[{'version': 'v1', 'created': 'Wed, 10 Feb 199...",2008-11-26,"[[Hayes, Mark E., , University College London]...",0.693147,NaN
1,1403.7138,Aigen Li,"Qi Li, S.L. Liang, Aigen Li (University of Mis...",Spectropolarimetric Constraints on the Nature ...,"5 pages, 2 figures; accepted for publication i...",None,10.1093/mnrasl/slu021,None,astro-ph.GA,http://arxiv.org/licenses/nonexclusive-distrib...,While it is well recognized that interstella...,"[{'version': 'v1', 'created': 'Thu, 27 Mar 201...",2015-06-19,"[[Li, Qi, , University of Missouri], [Liang, S...",2.197225,7.0
2,1405.5857,Michael Mortonson,"Michael J. Mortonson, Uro\v{s} Seljak",A joint analysis of Planck and BICEP2 B modes ...,"13 pages, 4 figures; submitted to JCAP; refere...",JCAP10(2014)035,10.1088/1475-7516/2014/10/035,None,astro-ph.CO gr-qc hep-ph hep-th,http://arxiv.org/licenses/nonexclusive-distrib...,We analyze BICEP2 and Planck data using a mo...,"[{'version': 'v1', 'created': 'Thu, 22 May 201...",2014-10-17,"[[Mortonson, Michael J., ], [Seljak, Uroš, ]]",4.812184,188.0
3,1807.01034,Evangelos Thomas Karamatskos,"Evangelos T. Karamatskos, Sebastian Raabe, Ter...",Molecular movie of ultrafast coherent rotation...,9 Figures,"Nat Commun 10, 3364 (2019)",10.1038/s41467-019-11122-y,None,physics.chem-ph physics.atom-ph quant-ph,http://arxiv.org/licenses/nonexclusive-distrib...,Recording molecular movies on ultrafast time...,"[{'version': 'v1', 'created': 'Tue, 3 Jul 2018...",2020-05-19,"[[Karamatskos, Evangelos T., ], [Raabe, Sebast...",1.945910,8.0
4,1905.05921,Juanjuan Gu,Juanjuan Gu and Yun Jing,A Modified Mixed Domain Method for Modeling Ac...,None,None,10.1121/10.0001454,None,physics.med-ph physics.comp-ph,http://arxiv.org/licenses/nonexclusive-distrib...,"In this paper, phase correction and amplitud...","[{'version': 'v1', 'created': 'Wed, 15 May 201...",2020-07-15,"[[Gu, Juanjuan, ], [Jing, Yun, ]]",0.000000,NaN


In [3]:
df_train.shape

(851524, 16)

In [4]:
df = df_train[['id', 'title', 'abstract', 'doi_cites']]
df['text'] = df['title'] + df['abstract']
df = df[['id', 'text', 'doi_cites']]
df.head()

<ipython-input-4-fad0058e69fc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['title'] + df['abstract']


,id,text,doi_cites
0,hep-ph/9902295,Heavy-Flavour Production at HERA We review th...,0.693147
1,1403.7138,Spectropolarimetric Constraints on the Nature ...,2.197225
2,1405.5857,A joint analysis of Planck and BICEP2 B modes ...,4.812184
3,1807.01034,Molecular movie of ultrafast coherent rotation...,1.945910
4,1905.05921,A Modified Mixed Domain Method for Modeling Ac...,0.000000


In [5]:
df.shape

(851524, 3)

In [18]:
from transformers import AutoTokenizer

def encode_texts(texts, maxlen=312):
    tokenizer = AutoTokenizer.from_pretrained(MOLDEL_NAME)
    enc_di = tokenizer.batch_encode_plus(
        list(texts), 
        truncation=True,
        padding='longest',
        max_length=maxlen
    )
    return enc_di['input_ids'], enc_di['attention_mask']


# 一気にやるとメモリ枯渇するので分割
num = 100000
ids, att = encode_texts(df['text'].values[:num])
for i in range(num, len(df), num):
    print(i)
    end_num = i + num
    if num + i > len(df):
        end_num = len(df)
    ids_temp, att_temp = encode_texts(df['text'].values[i:end_num])
    ids = np.vstack([ids, ids_temp])
    att = np.vstack([att, att_temp])
len(ids), len(att)

10000


(15117, 15117)

In [20]:
len(ids[0]), len(att[0])

(312, 312)

In [21]:
def create_model():
    """ Creates the model. It is composed of the XLNet main block and then
    a classification head its added
    """
    ids = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32, name="ids")
    att = tf.keras.layers.Input((MAX_LEN,), dtype=tf.int32, name="atttention_mask")

    bert_model = TFRobertaModel.from_pretrained(MOLDEL_NAME)
    x = bert_model(ids, attention_mask=att)[0]
    #x = tf.keras.layers.Dropout(0.1, name="dropout")(x[0]) 
    #x = tf.keras.layers.Conv1D(1,1, name="conv1d")(x)
    x = tf.squeeze(x[:, -1:, :], axis=1)
    #x = tf.keras.layers.Flatten()(x)
    #doc_encoding = tf.keras.layers.Dropout(.1)(doc_encoding)
    outputs = tf.keras.layers.Dense(1, name='outputs')(x)

    # Compile model
    model = tf.keras.Model(inputs=[ids, att], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=2e-5), loss='mse', metrics='mse')

    return model

In [22]:
model = create_model()
model.summary()

Some layers from the model checkpoint at roberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at roberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape       

In [23]:
layer_name = 'tf_op_layer_Squeeze_2'
hidden_layer_model = tf.keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
bert_vec = hidden_layer_model.predict([ids, att], batch_size=256, verbose=1)
bert_vec.shape

60/60 [==============================] - 82s 1s/step


(15117, 768)

In [24]:
np.save('bert_vec_cites_not_null.npz', bert_vec)

In [25]:
col_dict = {}
for i in range(768):
    col_dict[i] = f'roberta_vec_{i}'

df_result = pd.concat([df_train[['id']], pd.DataFrame(bert_vec).rename(columns=col_dict)], axis=1)
df_result.head()

,id,roberta_vec_0,roberta_vec_1,roberta_vec_2,roberta_vec_3,roberta_vec_4,roberta_vec_5,roberta_vec_6,roberta_vec_7,roberta_vec_8,roberta_vec_9,roberta_vec_10,roberta_vec_11,roberta_vec_12,roberta_vec_13,roberta_vec_14,roberta_vec_15,roberta_vec_16,roberta_vec_17,roberta_vec_18,roberta_vec_19,roberta_vec_20,roberta_vec_21,roberta_vec_22,roberta_vec_23,roberta_vec_24,roberta_vec_25,roberta_vec_26,roberta_vec_27,roberta_vec_28,roberta_vec_29,roberta_vec_30,roberta_vec_31,roberta_vec_32,roberta_vec_33,roberta_vec_34,roberta_vec_35,roberta_vec_36,roberta_vec_37,roberta_vec_38,roberta_vec_39,roberta_vec_40,roberta_vec_41,roberta_vec_42,roberta_vec_43,roberta_vec_44,roberta_vec_45,roberta_vec_46,roberta_vec_47,roberta_vec_48,roberta_vec_49,roberta_vec_50,roberta_vec_51,roberta_vec_52,roberta_vec_53,roberta_vec_54,roberta_vec_55,roberta_vec_56,roberta_vec_57,roberta_vec_58,roberta_vec_59,roberta_vec_60,roberta_vec_61,roberta_vec_62,roberta_vec_63,roberta_vec_64,roberta_vec_65,roberta_vec_66,roberta_vec_67,roberta_vec_68,roberta_vec_69,roberta_vec_70,roberta_vec_71,roberta_vec_72,roberta_vec_73,roberta_vec_74,roberta_vec_75,roberta_vec_76,roberta_vec_77,roberta_vec_78,roberta_vec_79,roberta_vec_80,roberta_vec_81,roberta_vec_82,roberta_vec_83,roberta_vec_84,roberta_vec_85,roberta_vec_86,roberta_vec_87,roberta_vec_88,roberta_vec_89,roberta_vec_90,roberta_vec_91,roberta_vec_92,roberta_vec_93,roberta_vec_94,roberta_vec_95,roberta_vec_96,roberta_vec_97,roberta_vec_98,roberta_vec_99,roberta_vec_100,roberta_vec_101,roberta_vec_102,roberta_vec_103,roberta_vec_104,roberta_vec_105,roberta_vec_106,roberta_vec_107,roberta_vec_108,roberta_vec_109,roberta_vec_110,roberta_vec_111,roberta_vec_112,roberta_vec_113,roberta_vec_114,roberta_vec_115,roberta_vec_116,roberta_vec_117,roberta_vec_118,roberta_vec_119,roberta_vec_120,roberta_vec_121,roberta_vec_122,roberta_vec_123,roberta_vec_124,roberta_vec_125,roberta_vec_126,roberta_vec_127,roberta_vec_128,roberta_vec_129,roberta_vec_130,roberta_vec_131,roberta_vec_132,roberta_vec_133,roberta_vec_134,roberta_vec_135,roberta_vec_136,roberta_vec_137,roberta_vec_138,roberta_vec_139,roberta_vec_140,roberta_vec_141,roberta_vec_142,roberta_vec_143,roberta_vec_144,roberta_vec_145,roberta_vec_146,roberta_vec_147,roberta_vec_148,roberta_vec_149,roberta_vec_150,roberta_vec_151,roberta_vec_152,roberta_vec_153,roberta_vec_154,roberta_vec_155,roberta_vec_156,roberta_vec_157,roberta_vec_158,roberta_vec_159,roberta_vec_160,roberta_vec_161,roberta_vec_162,roberta_vec_163,roberta_vec_164,roberta_vec_165,roberta_vec_166,roberta_vec_167,roberta_vec_168,roberta_vec_169,roberta_vec_170,roberta_vec_171,roberta_vec_172,roberta_vec_173,roberta_vec_174,roberta_vec_175,roberta_vec_176,roberta_vec_177,roberta_vec_178,roberta_vec_179,roberta_vec_180,roberta_vec_181,roberta_vec_182,roberta_vec_183,roberta_vec_184,roberta_vec_185,roberta_vec_186,roberta_vec_187,roberta_vec_188,roberta_vec_189,roberta_vec_190,roberta_vec_191,roberta_vec_192,roberta_vec_193,roberta_vec_194,roberta_vec_195,roberta_vec_196,roberta_vec_197,roberta_vec_198,roberta_vec_199,roberta_vec_200,roberta_vec_201,roberta_vec_202,roberta_vec_203,roberta_vec_204,roberta_vec_205,roberta_vec_206,roberta_vec_207,roberta_vec_208,roberta_vec_209,roberta_vec_210,roberta_vec_211,roberta_vec_212,roberta_vec_213,roberta_vec_214,roberta_vec_215,roberta_vec_216,roberta_vec_217,roberta_vec_218,roberta_vec_219,roberta_vec_220,roberta_vec_221,roberta_vec_222,roberta_vec_223,roberta_vec_224,roberta_vec_225,roberta_vec_226,roberta_vec_227,roberta_vec_228,roberta_vec_229,roberta_vec_230,roberta_vec_231,roberta_vec_232,roberta_vec_233,roberta_vec_234,roberta_vec_235,roberta_vec_236,roberta_vec_237,roberta_vec_238,roberta_vec_239,roberta_vec_240,roberta_vec_241,roberta_vec_242,roberta_vec_243,roberta_vec_244,roberta_vec_245,roberta_vec_246,roberta_vec_247,roberta_vec_248,...,roberta_vec_518,roberta_vec_519,roberta_vec_520,roberta_vec_521,roberta_vec_522,roberta_vec_523,roberta_vec_524,robert

In [32]:
df_result = df_result[df_result['roberta_vec_0'].isnull()==False]
df_result.shape

(15117, 769)

In [29]:
df_temp = pd.read_pickle('../data/train/robaerta_raw_vec.pickle')
df_temp = df_temp[df_temp['roberta_vec_0'].isnull()==False]
df_temp.shape

(836407, 769)

In [33]:
df_concat_result = pd.concat([df_result, df_temp])
df_concat_result.shape

(851524, 769)

In [34]:
df_result.to_pickle('../data/train/10_robaerta_raw_vec.pickle')

In [13]:
df_temp = pd.read_pickle('../data/train/10_robaerta_raw_vec.pickle')
df_temp.shape

(15117, 769)

In [16]:
df_temp2 = pd.read_pickle('../data/train/robaerta_raw_vec.pickle')
df_temp2 = df_temp2[df_temp2['roberta_vec_0'].isnull()==False]
df_temp2.shape

(836407, 769)

In [17]:
df_concat_result = pd.concat([df_temp, df_temp2])
df_concat_result.shape

(851524, 769)

In [18]:
df_concat_result.to_pickle('../data/train/10_robaerta_vec.pickle')